Carga de archivos

In [133]:
import pandas as pd
import io
import os
import matplotlib.pyplot as plt
import numpy as np
import copy as copy

In [134]:
# Ruta de los archivos
ruta = "./Datasets/"

In [135]:
def ls(ruta = "./Datasets"):
    return [arch.name for arch in os.scandir(ruta) if arch.is_file()]

In [136]:
# Archivos de origen predeterminados
origin = np.array(['Clientes.csv',
 'Compra.csv',
 'Gasto.csv',
 'Localidades.csv',
 'Proveedores.csv',
 'Sucursales.csv',
 'TiposDeGasto.csv',
 'Venta.csv'])
codificacion = np.array(['utf-8','utf-8','utf-8','utf-8','ANSI','utf-8','utf-8','utf-8'])


In [137]:
# Leer los archivos que existen en el directorio
archivos = np.array(ls())

In [138]:
# Comprobar que archivos existen
csv_existentes = np.array([False, False, False, False, False, False, False, False])
excel_existe = False

s = ''
for word in archivos:
    s += word

idx = 0
for frame_origin in origin :
    if frame_origin in s:
        csv_existentes[idx] = True
    idx+=1

# Comprobacion del excel
if 'CanalDeVenta.xlsx' in s:
    excel_existe = True

print(csv_existentes)

[ True  True  True  True  True  True  True  True]


In [139]:
# Cargamos las tablas csv en una lista
data = []
u = 0
for i in origin:
    if csv_existentes[u]:
        r = ruta+i
        g = open(r,encoding=codificacion[u])
        n = 1
        for i in g:
            #print(i)
            if ',' in i:
                #print('Delimiter coma')
                data.append(pd.read_csv(r,encoding=codificacion[u]))
            else:
                #print('delimiter punto y coma')
                data.append(pd.read_csv(r,delimiter=';',encoding=codificacion[u]))
            break
        #print(u,r)
        g.close()
    u+=1

print(archivos)


['CanalDeVenta.xlsx' 'Clientes.csv' 'Compra.csv' 'Gasto.csv'
 'Localidades.csv' 'Proveedores.csv' 'Sucursales.csv' 'TiposDeGasto.csv'
 'Venta.csv']


In [140]:
archivos = copy.copy(origin[csv_existentes == True])

In [141]:
# Actualizamos la variable archivos

# Cargamos el excel, si existe
if excel_existe:
    data.append(pd.read_excel(ruta+'CanalDeVenta.xlsx'))
    archivos = np.append(archivos,['CanalDeVenta.xls'])
print(archivos)

['Clientes.csv' 'Compra.csv' 'Gasto.csv' 'Localidades.csv'
 'Proveedores.csv' 'Sucursales.csv' 'TiposDeGasto.csv' 'Venta.csv'
 'CanalDeVenta.xls']


In [142]:
data[-1]

,CODIGO,DESCRIPCION
0,1,Telefónica
1,2,OnLine
2,3,Presencial


Homogenizar el nombre de los campos ID de cada tabla

In [143]:
for idx, table in enumerate(data):
    name = 'ID_'+archivos[idx][:-4]
    data[idx].rename(columns={table.columns[0] : name}, inplace=True)

Homogenizar el nombre de los campos de latitud y longitud

In [144]:
data[0].rename(columns={data[0].columns[-3] : 'Longitud',data[0].columns[-2] : 'Latitud'}, inplace=True)
data[3].rename(columns={data[3].columns[2] : 'Longitud',data[3].columns[1] : 'Latitud'}, inplace=True)
data[-4].rename(columns={data[-4].columns[6] : 'Longitud',data[-4].columns[5] : 'Latitud'}, inplace=True)

Convertir las columnas de latitud y longitud a float

In [145]:
tablas = [0,3,-4]
campos = ['Longitud','Latitud']
for df in tablas:
    for col in campos:
        #df.Y = df.Y.replace(np.nan,80.5)
        #df.Y = df.Y.astype(str)
        data[df][col] = data[df][col].replace(np.nan,80.5).copy()
        data[df][col] = data[df][col].astype(str)

    #for idx in df.index:
    #    if type(df.Y.iloc[idx]) != float:
    #        df.Y.iloc[idx] = float(df.Y.iloc[idx].replace(',','.'))
        for idx in data[df].index:
            try:
                if type(data[df][col].iloc[idx]) != float:
                    data[df][col].iloc[idx] = float(data[df][col].iloc[idx].replace(',','.'))
            except:
                print(df,col,idx)
                break
        
        data[df][col] = data[df][col].astype(float)

C:\Users\exort\AppData\Local\Temp\ipykernel_19056\1552489187.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[df][col].iloc[idx] = float(data[df][col].iloc[idx].replace(',','.'))


Tabla del precio medio de los productos de la tabla Compra y Ventas

In [162]:
# Tablas base
t = np.array(['Compra.csv','Venta.csv'])

for tablas in t:
    if tablas in archivos:
        # Valores unicos en el IdProducto en la tabla compras
        productos = data[1].IdProducto.unique()
        precio_medio = np.zeros((data[1].IdProducto.unique().shape[0],1))

        # Obtenemos el precio medio
        for idx, product in enumerate(productos):
            precio_medio[idx] = data[1].Precio[data[1].IdProducto == product].median()

        # Sustituimos el nulo por el precio medio
        for i in data[1][data[1].Precio.isnull() == True].index:
            data[1].Precio.iloc[i] = precio_medio[productos == data[1].IdProducto.iloc[i]]

In [165]:
data[1][data[1].Precio.isnull() == True]

,ID_Compra,Fecha,Fecha_Año,Fecha_Mes,Fecha_Periodo,IdProducto,Cantidad,Precio,IdProveedor


In [163]:
archivos

array(['Clientes.csv', 'Compra.csv', 'Gasto.csv', 'Localidades.csv',
       'Proveedores.csv', 'Sucursales.csv', 'TiposDeGasto.csv',
       'Venta.csv', 'CanalDeVenta.xls'], dtype='<U16')

In [164]:
for idx, table in enumerate(data):
    print(list(table.columns))

['ID_Clientes', 'Provincia', 'Nombre_y_Apellido', 'Domicilio', 'Telefono', 'Edad', 'Localidad', 'Longitud', 'Latitud', 'col10']
['ID_Compra', 'Fecha', 'Fecha_Año', 'Fecha_Mes', 'Fecha_Periodo', 'IdProducto', 'Cantidad', 'Precio', 'IdProveedor']
['ID_Gasto', 'IdSucursal', 'IdTipoGasto', 'Fecha', 'Monto']
['ID_Localidades', 'Latitud', 'Longitud', 'departamento_id', 'departamento_nombre', 'fuente', 'id', 'localidad_censal_id', 'localidad_censal_nombre', 'municipio_id', 'municipio_nombre', 'nombre', 'provincia_id', 'provincia_nombre']
['ID_Proveedores', 'Nombre', 'Address', 'City', 'State', 'Country', 'departamen']
['ID_Sucursales', 'Sucursal', 'Direccion', 'Localidad', 'Provincia', 'Latitud', 'Longitud']
['ID_TiposDeGasto', 'Descripcion', 'Monto_Aproximado']
['ID_Venta', 'Fecha', 'Fecha_Entrega', 'IdCanal', 'IdCliente', 'IdSucursal', 'IdEmpleado', 'IdProducto', 'Precio', 'Cantidad']
['ID_CanalDeVenta', 'DESCRIPCION']
